In [ ]:
import sys
def reload_modules(*modules):
    for module in modules:
        if module.__name__ in sys.modules:
            importlib.reload(module)
        else:
            print(f"Module {module.__name__} is not currently imported.")
import numpy as np
import copy
from PIL import Image
import open3d as o3d
import cv2
import importlib
import matplotlib.pyplot as plt
from magpie_perception import pcd
from magpie_perception import utils
from magpie_perception import utils
from magpie_perception.utils import label_wrist_image, find_object
from magpie_control import realsense_wrapper as real
importlib.reload(pcd, utils, real, ur5)

# from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'magpie_perception'

In [2]:
devices = real.poll_devices()
print(devices)
wrist = real.RealSense(fps=15, w=640, h=480, device_name="D405")
wrist.initConnection(device_serial=devices['D405'])
wkspc = real.RealSense(zMax=5, fps=6, w=640, h=480, device_name="D435")
wkspc.initConnection(device_serial=devices['D435'])

{'D435': '832412070344', 'D405': '126122270157'}
There are 2 available devices!
There are 2 available devices!


In [48]:
pcd_wrist, rgbd_wrist = wrist.getPCD()
pcd_wkspc, rgbd_wkspc = wkspc.getPCD()

img_wrist = np.array(rgbd_wrist.color)
img_wkspc = np.array(rgbd_wkspc.color)

In [176]:
def label_wrist_image(wrist_img):
    '''
    labels wrist image with overlaid x,y,z axes in line with F/T sensor
    '''
    h, w, c = wrist_img.shape

    x_color = (0, 255, 0) # green
    y_color = (0, 0, 255) # blue
    z_color = (255, 0, 0) # red

    # Define axis line lengths (adjust as needed)
    axis_length = min(w, h) // 3

    # Calculate starting point for axes (center of the image)
    center_x, center_y = w // 2, h // 2

    # Draw X-axis
    cv2.arrowedLine(wrist_img, (center_x, center_y), (center_x + axis_length, center_y), x_color, 6)
    cv2.putText(wrist_img, "+X", (center_x + axis_length - 15, center_y -20), cv2.FONT_HERSHEY_SIMPLEX, 2, x_color, 6)

    # Draw Y-axis
    cv2.arrowedLine(wrist_img, (center_x, center_y), (center_x, center_y - axis_length), y_color, 6)
    cv2.putText(wrist_img, "-Y", (center_x - 15, center_y - axis_length - 5), cv2.FONT_HERSHEY_SIMPLEX, 2, y_color, 6)

    # Draw Z-axis (pointing towards the viewer, represented as a circle)
    cv2.circle(wrist_img, (center_x, center_y), 9, z_color, -1)  # Filled circle
    cv2.putText(wrist_img, "+Z", (center_x - 45, center_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 2, z_color, 6)

    return wrist_img

img_wrist_rotated = cv2.rotate(img_wrist, cv2.ROTATE_90_COUNTERCLOCKWISE)
wrist_img_labeled = label_wrist_image(img_wrist_rotated.copy()) # Use .copy() to avoid modifying the original
# img_wkspc_resized = cv2.resize(img_wkspc, (img_wrist_rotated.shape[1], img_wrist_rotated.shape[0]))
import imutils
img_wkspc_resized = imutils.resize(img_wkspc, height=img_wrist_rotated.shape[0])
final_img = np.hstack((img_wkspc_resized, wrist_img_labeled))

# save img
fn = "axes6.png"
cv2.imwrite(fn, cv2.cvtColor(final_img, cv2.COLOR_RGB2BGR))

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.imshow(final_img)

NameError: name 'cv2' is not defined

In [ ]:
# with help from: https://github.com/google-gemini/cookbook/blob/main/examples/Spatial_understanding_3d.ipynb
import os
from PIL import Image
from google import genai
import openai
from dotenv import load_dotenv
import instructor
from force_prediction_prompts import initial_prompt, feedback_prompt
from magpie_prompts.prompts import sf_force_thinker
from magpie_prompts import conversation
from magpie_prompts.conversation import openai_encode_image, build_messages, send_message
from magpie_prompts.prompts import sf_grasp_selection
import importlib
importlib.reload(sf_force_thinker)
importlib.reload(conversation)
importlib.reload(sf_grasp_selection)

user_query = "grab the scissors"
user_query = "grab the screwdriver"
user_query = "open the box"
user_query = "close the box"

grasp_prompt = sf_grasp_selection.grasp_prompt.format(**{"user_query": user_query})
img = Image.fromarray(np.array(rgbd_wrist.color))
image_response = gemini_client.models.generate_content(
    model=GEMINI_MODEL_ID,
    contents=[grasp_prompt, img],
)

image_response.text.split("\n")[-1]
image_response.text.split("\n")[-1].split(": ")[-1]

In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
OPENAI_API_KEY = os.getenv('CORRELL_API_KEY')
GEMINI_MODEL_ID = "gemini-2.0-flash" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true}
# OPENAI_MODEL_ID = "gpt-4o-mini"
OPENAI_MODEL_ID = "gpt-4o"

gemini_client = genai.Client(api_key=GEMINI_API_KEY)
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
force_thinker = sf_force_thinker.PromptForceThinker()

In [174]:
model_type = "gemini"
# model_type = "openai"
client, model = None, None
if model_type == "openai":
    client = openai_client
    model = OPENAI_MODEL_ID
elif model_type == "gemini":
    client = gemini_client
    model = GEMINI_MODEL_ID
# img = Image.open('axes.png').convert("RGB")
# img = Image.open('axes4_obj-box-lid_motion-close.png').convert("RGB")
img = Image.open('axes3_obj-box-lid_motion-open.png').convert("RGB")

# obj = "small servo motor"
# prompt_config = {
# "obj": "small servo motor",
# "task": f"slide the {obj} rapidly up the table for 10 cm"
# }
obj = "case lid handle"
prompt_config = {
"obj": obj,
"task": f"open the case by the {obj}"
}

# prompt = initial_prompt.format(**prompt_config)
messages = []
prompt = sf_force_thinker.prompt_motion_thinker.format(**prompt_config)
messages = build_messages(prompt, img, messages=messages, model_type=model_type)
response = send_message(client, model, messages, model_type=model_type)
motion_plan = force_thinker.process_thinker_response(response)


# print(response)
# print(response.split("```"))
# print(response.output_text)


In [175]:
print(motion_plan)

{'motion_direction': [1, -1, 0], 'motion_goal': [0.05, 0.05, 0.0], 'force': [1.0, 0.75, 0.25], 'grasp_force': 2.5, 'duration': 2.0}


In [157]:
print(response)

[start of motion plan]
The right image is labeled with the axes of motion, which do not necessarily correspond with the world axes.
The task is to open the case by the case lid handle while grasping the case lid handle.
The provided image confirms {DESCRIPTION: a blue plastic case sits on a grid surface. The case has a handle on the lid. A robot end-effector is positioned above the handle ready to grasp it. The task is to open the case by lifting the lid using the handle.}.
The green axis pointing to the right of the image is the positive X-axis, corresponding {DESCRIPTION: to the primary direction of the lid opening. The lid will need to move in the positive X direction to open the case.}.
The blue axis pointing up the image is the negative Y-axis, corresponding {DESCRIPTION: to a potential force component needed to maintain the grip on the handle and overcome any vertical friction or locking mechanisms. Minimal motion along Y is expected unless there is a latch.}.
The red dot pointin

In [ ]:
import ast
# remove strings from list that begin with # or are empty
motion_plan_string = str(response).split("python")[-1].split("```")[0].split('\n')
motion_plan_string = [x for x in motion_plan_string if x and not x.startswith("#")]
motion_plan_string = [x.strip() for x in motion_plan_string]
# separate variable names and values from strings, make motion_plan dict
motion_plan = {x.split("=")[0].strip(): ast.literal_eval(x.split("=")[1]) for x in motion_plan_string}
motion_plan
'''
['motion_direction = [1, -1, 0]',
 'motion_goal = [3.0, 6.0, 0.0]',
 'force = [3.5, 7.5, 0.0]',
 'grasp_force = 4.5',
 'duration = 5.0']
'''




{'motion_direction': [1, -1, 0],
 'motion_goal': [0.15, 0.0, 0.0],
 'force': [3.0, 1.0, 0.5],
 'grasp_force': 3.5,
 'duration': 2.0}

In [ ]:
from magpie_prompts.schemas import force_prediction_schema as fps
import importlib
importlib.reload(fps)

# input=[{
#     "role": "user",
#     "content": [
#         {"type": "text", "text": f"{prompt}"},
#         {
#             "type": "image_url",
#             "image_url": f"data:image/jpeg;base64,{openai_encode_image(img)}",
#         },
#     ],
# }]
input=[{
    "role": "user",
    "content": [
        prompt,
        instructor.Image.from_raw_base64(openai_encode_image(img)),
    ],
}]

openai_client_structured = instructor.from_openai(client=openai_client)
response = openai_client_structured.chat.completions.create(
    messages=input,
    response_model=force_prediction_schema.MotionForcePlan,
    model=model,
)

In [52]:
response.goal_force


AttributeError: 'MotionForcePlan' object has no attribute 'goal_force'

In [40]:
print(response.output_text)


To complete the task of sliding the small servo motor rapidly up the table for 10 cm, we'll estimate the motion direction, displacement, required forces, time, and grasping force.

### **Step 1: Initial Motion and Force Plan**

1. **Direction of Motion:**  
   Since we want to slide the servo motor up the table, this corresponds to moving in the +X direction based on the labeled axes.
   ```python
   direction = [1, 0, 0]  # +X direction
   ```

2. **Goal Position:**  
   The target displacement is 10 cm in the +X direction.
   ```python
   goal_position = [0.1, 0, 0]  # 0.1 meters in +X
   ```

3. **Goal Force:**  
   - **Mass Estimation:** Assume the small servo motor has a mass of approximately 0.05 kg.  
   - **Friction:** Coefficient of friction is assumed to be around 0.3 for a typical surface.  
   - **Force Calculation:**
     - Normal Force, \( F_N = m \cdot g = 0.05 \cdot 9.81 = 0.4905 \) N  
     - Frictional Force, \( F_f = \mu \cdot F_N = 0.3 \cdot 0.4905 \approx 0.1472 \)

In [21]:
print(image_response.text)

Okay, I understand the setup and the requested format. Here's the analysis for the desired motion:

**Desired motion description:** Grasp the {obj}

**Analysis:**

1.  **Coarse Motion:**
    *   X: 0 (No significant motion along the X-axis is required for grasping. The gripper is aligned on that axis)
    *   Y: -1 (The gripper needs to move down towards the object.)
    *   Z: 0 (No significant motion along the Z-axis is required for grasping. The gripper is aligned on that axis)

2.  **Fine Motion:**
    *   X: 0.00 (No significant motion)
    *   Y: -0.05 (Estimate the distance to move down based on the visible gap between the gripper and the object. An estimated 5cm should be enough)
    *   Z: 0.00 (No significant motion)

3.  **Coarse Forces:**
    *   X: 0 (No significant forces along the X-axis are expected initially)
    *   Y: 1 (A positive force (upwards) will be necessary to counteract gravity once the object is grasped)
    *   Z: 0 (No significant forces along the Z-axis 